In [1]:
# Transformers installation
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.4 MB/s eta 0:00:00


## Loading Data

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd

df_train= pd.read_csv('/content/train.csv')
df_dev= pd.read_csv('/content/dev.csv')

df_train.rename(columns= {'initial_request': 'text','clarification_need': 'label'}, inplace= True)
df_dev.rename(columns= {'initial_request': 'text','clarification_need': 'label'}, inplace= True)

df_train['label']= [df_train['label'][i]-1 for i in range(len(df_train['label']))]  #since model uses labels 0-3
df_dev['label']= [df_dev['label'][i]-1 for i in range(len(df_dev['label']))]


train_data = Dataset.from_pandas(df_train) #.select(range(184))
dev_data = Dataset.from_pandas(df_dev) #.select(range(48))

In [3]:
train_data[0]

{'text': 'Tell me about Obama family tree.', 'label': 1}

## Tokenizing Data

In [4]:
from transformers import AutoTokenizer, OpenAIGPTForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gpt2")
from transformers import AutoModelForSequenceClassification, AutoModel

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=4)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id
# def tokenize_function(examples):

#     return tokenizer(examples["text"], padding="max_length", truncation=True) #padding="max_length", add this argument if needed

def tokenize_function(examples):

    tnizer = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # model.resize_token_embeddings(len(tokenizer))
    return tnizer #padding="max_length", add this argument if needed

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_dev = dev_data.map(tokenize_function, batched=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using pad_token, but it is not set yet.


Map:   0%|          | 0/187 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [5]:
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch")

tokenized_dev = tokenized_dev.remove_columns(["text"])
tokenized_dev = tokenized_dev.rename_column("label", "labels")
tokenized_dev.set_format("torch")

### Dataloader

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train, batch_size=8)
eval_dataloader = DataLoader(tokenized_dev, batch_size=8)

### Model

In [7]:
import torch

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoModel

# model = AutoModelForSequenceClassification.from_pretrained("t5", num_labels=4)

In [8]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [9]:
from transformers import get_scheduler

num_epochs = 50
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [10]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")
model.to(device)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid

### Training

In [11]:
import os
from datasets import load_metric
from tqdm.auto import tqdm

In [12]:
def train_one_batch(model, batch, optimizer, lr_scheduler):
  batch = {k: v.to(device) for k, v in batch.items()}
  outputs = model(**batch)
  loss = outputs.loss
  loss.backward()

  optimizer.step()
  lr_scheduler.step()
  optimizer.zero_grad()

  return loss, model, optimizer, lr_scheduler


def train_one_epoch(model, train_dataloader, optimizer, lr_scheduler):
  model.train()
  loss = 0
  progress_bar = tqdm(range(len(train_dataloader)))

  for batch in train_dataloader:
    running_loss, model, optimizer, lr_scheduler = train_one_batch(model, batch, optimizer, lr_scheduler)
    loss += running_loss
    progress_bar.update(1)

  loss = loss / len(train_dataloader)
  return loss, model, optimizer, lr_scheduler
  

def get_val_loss(model, val_dataloader):
  model.eval()
  loss = 0

  metric = load_metric('f1')
  with torch.no_grad():
    for batch in val_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      running_loss = outputs.loss
      loss += running_loss

      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      metric.add_batch(predictions=predictions, references=batch["labels"])
  
  f1_score = metric.compute(average= 'weighted')
  
  return loss / len(val_dataloader), f1_score

In [13]:
# num_epochs=10
for epoch in range(num_epochs):
  print(f'Epoch: [{epoch+1} / {num_epochs}]:')
  t_loss, model, optimizer, lr_scheduler = train_one_epoch(model, train_dataloader, optimizer, lr_scheduler)
  v_loss, f1_score = get_val_loss(model, eval_dataloader)
  # print
  print(f"\tLoss -> Train: {t_loss:.5f} | Val: {v_loss} | F1 Score: {f1_score['f1']}")
  # save
  # DST Folder
  DST = '/content/checkpoints'
  if not os.path.isdir(DST):
    os.makedirs(DST)
  path = os.path.join(DST, f'epoch_{epoch+1}.pth')
  torch.save(model.state_dict(), path)

Epoch: [1 / 50]:


  0%|          | 0/24 [00:00<?, ?it/s]

<ipython-input-12-c4fed72155ad>:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('f1')


	Loss -> Train: 1.67422 | Val: 1.1391878128051758 | F1 Score: 0.2836122448979592
Epoch: [2 / 50]:


  0%|          | 0/24 [00:00<?, ?it/s]

...

	Loss -> Train: 0.00279 | Val: 1.7450053691864014 | F1 Score: 0.40662623599208964
Epoch: [49 / 50]:


  0%|          | 0/24 [00:00<?, ?it/s]

	Loss -> Train: 0.00301 | Val: 1.7450342178344727 | F1 Score: 0.40662623599208964
Epoch: [50 / 50]:


  0%|          | 0/24 [00:00<?, ?it/s]

	Loss -> Train: 0.00335 | Val: 1.744935393333435 | F1 Score: 0.40662623599208964


### Testing

In [14]:
df_test= pd.read_csv('/content/test_with_labels.csv')

df_test.rename(columns= {'initial_request': 'text','clarification_need': 'label'}, inplace= True)
df_test['label']= [df_test['label'][i]-1 for i in range(len(df_test['label']))]
test_data = Dataset.from_pandas(df_test) #.select(range(184))
tokenized_test = test_data.map(tokenize_function, batched=True)

tokenized_test = tokenized_test.remove_columns(["text"])
tokenized_test = tokenized_test.rename_column("label", "labels")
tokenized_test.set_format("torch")
test_dataloader = DataLoader(tokenized_test, batch_size=8)

# model = TheModelClass(*args, **kwargs)
# modelnew = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=4)
# modelnew.to(device)
# modelnew.load_state_dict(torch.load('/content/checkpoints/epoch_8.pth'))
# modelnew.eval()
# modelnew= torch.load('/content/checkpoints/epoch_8.pth',weights_only= True)

test_loss, f1_score = get_val_loss(model, test_dataloader)

print(f"Test loss is: {test_loss:.5f}, F1 score is: {f1_score['f1']:.5f}")

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Test loss is: 2.00375, F1 score is: 0.50007
